# load bitstream

In [42]:

from pynq import Overlay
import numpy as np
from pynq import allocate
import time

ol = Overlay('top_v2-4.bit')

# allocate buf

In [45]:

# u4: 32-bit unsigned integers
input_buffer = allocate(shape=(25000000,), dtype='u4')

input_buffer[:256] = np.random.randint(10, size=256)
input_buffer.flush()

input_buffer.device_address, input_buffer[:26]

(378535936,
 PynqBuffer([1, 6, 4, 3, 9, 2, 7, 4, 5, 9, 9, 8, 9, 4, 0, 5, 7, 1, 5, 1, 0,
             6, 0, 6, 6, 1], dtype=uint32))

In [46]:
np.sum(input_buffer[:256])

PynqBuffer(1097, dtype=uint32)

# transfer tpu program

In [43]:


binprog = [
            (0b00000001 << 56) | (0 << 24) | (16 << 8), # read ddr, start addr: 0, nburst: 16.
            (0b00111111 << 56)                          # end.
          ]

for i, inst in enumerate(binprog):
    print(f"{i}:", '0x{0:0{1}X}'.format(inst, 16))
    
binprog = np.array(binprog, dtype=np.uint64)

binprog = np.frombuffer(binprog, np.uint32)

np.copyto(ol.axi_bram_ctrl_0.mmio.array[:len(binprog)], binprog)


0: 0x0100000000001000
1: 0x3F00000000000000


In [41]:
binprog

array([      4096,   16777216,          0, 1056964608], dtype=uint32)

In [39]:
ol.qt3_tpu_v1_0.mmio.array[3] = 8

In [40]:
ol.qt3_tpu_v1_0.mmio.array[7], ol.qt3_tpu_v1_0.mmio.array[9]

(1056964608, 0)

# run

In [47]:

ol.qt3_tpu_v1_0.mmio.array[1] = np.uint32(input_buffer.device_address) # DDR_BASEADDR_REG
time.sleep(0.2)

ol.qt3_tpu_v1_0.mmio.array[0] = np.uint32(1) # START_REG
time.sleep(0.2)

ol.qt3_tpu_v1_0.mmio.array[0] = np.uint32(0) # START_REG
time.sleep(0.2)

print("reg2: ", ol.qt3_tpu_v1_0.mmio.array[2])
for i in range(5, 10):
    print(f"reg{i}: ", ol.qt3_tpu_v1_0.mmio.array[i])

reg2:  1097
reg5:  617
reg6:  256
reg7:  0
reg8:  16
reg9:  16
